<a href="https://colab.research.google.com/github/Word-Wizards/word2vac/blob/Word2Vac/NLP_Dhruvi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#mount my google drive
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
import pandas as pd
import os

In [ ]:
df = pd.read_json('/content/gdrive/MyDrive/Data/dataset.gz', lines=True, compression='gzip')

In [ ]:
df.head()

,id,source,version,added,created,text
0,254721023,s2ag/valid,v1,2022-12-16T16:07:05.559Z,2022-12-14T00:00:00.000Z,Investigation Some Characteristics of Biocompo...
1,254200673,s2ag/valid,v1,2022-12-04T16:21:43.441Z,2022-12-02T00:00:00.000Z,Modelling Techniques for Conjunctive Water Man...
2,252643161,s2ag/valid,v1,2022-10-01T15:18:53.788Z,2022-12-01T00:00:00.000Z,Complete Targets Coverage in Energy Harvesting...
3,244852373,s2ag/valid,v1,2021-12-04T16:21:39.980Z,2022-12-01T00:00:00.000Z,Nonorthogonal Multiple Access Enabled Two-Way ...
4,254774363,s2ag/valid,v1,2022-12-17T16:19:10.635Z,2022-12-14T00:00:00.000Z,106 A CASE OF COMPLETE ATRIOVENTRICULAR BLOCK ...


In [ ]:
df_train=df[:3000]


In [ ]:
df_train.shape

(3000, 6)

In [ ]:
df_test=df[3000:4000]

In [ ]:
df_test.shape

(1000, 6)

In [ ]:
import os
import re
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords

In [ ]:
# Initialize the TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(lowercase=False, stop_words=stopwords.words('english'))

LookupError: ignored

In [ ]:
import re
import string
import pandas as pd
import numpy as np
import math
from numpy import array
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.cluster import KMeans
from collections import Counter
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize, pos_tag

nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
# Pre-processing function to apply the specified cleanup
def preprocess_text(text):
    # Remove URLs
    text = re.sub(r'http\S+', '', text)

    # Remove bullets, enumerations, hyphens, and punctuation
    text = re.sub(r'[\•\-\d]+', ' ', text)
    text = re.sub(r'[\'\"\,\.\!\?\:\;\(\)\[\]\{\}\<\>]', ' ', text)

    # Convert numerical values to text
    text = re.sub(r'\d+', lambda x: num2words(x.group()), text)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    words = word_tokenize(text.lower())
    words = [word for word in words if word not in stop_words]

    # Reconstruct sentences
    text = " ".join(words)

    return text


In [ ]:
data = [doc for doc in df_train['text'].tolist()]

In [ ]:
def preprocess_text1(text):
    text = re.sub(r'\d+' , '', text)
    text = remove_punctuation(text)
    text = text.lower()
    text = text.strip()
    text = re.sub(r'\b\w{1,2}\b', '', text)
    tokens = toknizing(text)
    pos_map = {'J': 'a', 'N': 'n', 'R': 'r', 'V': 'v'}
    pos_tags = pos_tag(tokens)
    lemmatiser = WordNetLemmatizer()
    tokens = [lemmatiser.lemmatize(t.lower(), pos=pos_map.get(p[0], 'v')) for t, p in pos_tags]
    return tokens

In [ ]:
def remove_punctuation(text):
     punctuations = '''!()-[]{};:'"\,<>./?@#$%^+&*_~'''
     no_punct = ""
     for char in text:
        if char not in punctuations:
            no_punct = no_punct + char
     return no_punct

In [ ]:
def toknizing(text):
  stop_words = set(stopwords.words('english'))
  tokens = word_tokenize(text)
  result = [i for i in tokens if not i in stop_words]
  return result

In [ ]:
train_sentences = [preprocess_text1(doc) for doc in df_train['text'].tolist()]
test_sentences = [preprocess_text1(doc) for doc in df_test['text'].tolist()]

#returning sentences instead of tokens

In [ ]:
# Tokenize and preprocess the documents into sentences
def tokenize_and_preprocess(document):
    sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?|\!)\s', document)
    sentences = [preprocess_text(sentence) for sentence in sentences if sentence.strip()]
    return sentences

In [ ]:
train_sentences = [tokenize_and_preprocess(doc) for doc in df_train['text'].tolist()]
test_sentences = [tokenize_and_preprocess(doc) for doc in df_test['text'].tolist()]

In [ ]:
train_sentences[0]

['investigation characteristics biocomposites coating biomedical implants surface vital function tissue response presence foreign material field body implants',
 'surface modification coatings adjusted provide highest service performance lowest cost',
 'coatings increase corrosion resistance reducing metal ion corrosion product migration body',
 'fabricated polymer based bio composite coatings blending chitosan chi alginate alg nanoparticles tio nb dip coating onto l stainless steel substrate',
 'coatings ’ surface morphology phases studied using fesem ftir analysis',
 'wettability behavior coated samples also studied investigating contact wetting attributes',
 'antibacterial activity functionalized coatings determined',
 'ftir results showed blending chi alg nanoparticles excellent obvious differences spectra changes structures polymer matrices observed',
 'sem results demonstrated coating layers uniform homogeneous crack free l stianless steel substrate using tio nb nano particles',


In [ ]:
combined_list = [item for sublist in train_sentences for item in sublist]

In [ ]:
combined_list[0]

'investigation characteristics biocomposites coating biomedical implants surface vital function tissue response presence foreign material field body implants'

In [ ]:
len(combined_list)

26747

In [ ]:
import os
import re
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords

In [ ]:
# Initialize the TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(lowercase=False, stop_words=stopwords.words('english'))

In [ ]:
# TD-IDF Matrix
X = tfidf_vectorizer.fit_transform(combined_list)

# extracting feature names
tfidf_tokens = tfidf_vectorizer.get_feature_names_out()

In [ ]:
tfidf_tokens

array(['_group', 'aa', 'aaa', ..., 'ﬂow', 'ﬂuid', '𝛿𝑡'], dtype=object)

In [ ]:
import pandas as pd

result = pd.DataFrame(
    data=X.toarray(),
    index=[X for X in combined_list],
    columns=tfidf_tokens
)

result

,_group,aa,aaa,aacr,aada,aapc,aaron,aas,aats,aav,...,ﬁnal,ﬁnd,ﬁrst,ﬁxed,ﬁxes,ﬂight,ﬂocks,ﬂow,ﬂuid,𝛿𝑡
investigation characteristics biocomposites coating biomedical implants surface vital function tissue response presence foreign material field body implants,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
surface modification coatings adjusted provide highest service performance lowest cost,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
coatings increase corrosion resistance reducing metal ion corrosion product migration body,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
fabricated polymer based bio composite coatings blending chitosan chi alginate alg nanoparticles tio nb dip coating onto l stainless steel substrate,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
coatings ’ surface morphology phases studied using fesem ftir analysis,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
negative significant correlation found e gfr ckd prognosis p,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
additionally also significant correlations plasma ace il ckd prognosis p = r = p = r = respectively,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
significant positive correlation also found plasma ace il p = r =,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
conclusions significant correlation plasma ace il levels ckd prognosis,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
